In [1]:
import os
import cv2
import numpy as np
import tensorflow as tf
import skimage
import time

In [2]:
def preProcessFromPath(image, listImg, listAns, category):
    assert type(image) != None
    assert image.shape == (1080, 1920, 3)
    image = cv2.resize(image, (28, 28))
    assert type(image) == np.ndarray
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image = image.astype('float32')
    image = image/255.0
    listImg.append(image)
    listAns.append(category)
    return listImg, listAns

def captureImgWithPath(path, category):
    fileList = os.listdir(path)
    imgList = []
    ansList = []
    for name in fileList:
        image_full_path = path + '/' + name
        img = cv2.imread(image_full_path)
        imgList, ansList = preProcessFromPath(img, imgList, ansList, category)
    return imgList, ansList

def captureImgWithCam(img):
    if img is not None:
        imgList, _ = preProcessFromPath(img, [], [], 0)
        imgArr = np.array(imgList)
        return imgArr
    else:
        assert(0)
    return None

def unison_shuffled_copies(a, b):
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]

def computeNextState(listMsd, previousState, threshold):
    if len(listMsd) > 4:
        listMsd = listMsd[1 : ]
    avg = sum(listMsd)/len(listMsd)
    if previousState == 'empty' and avg > threshold:
        return 'inMotionForward', listMsd
    elif previousState == 'inMotionForward' and avg < threshold:
        return 'filled', listMsd
    elif previousState == 'filled' and avg > threshold:
        return 'inMotionBackward', listMsd
    elif previousState == 'inMotionBackward' and avg < threshold:
        return 'empty', listMsd
    else:
        return previousState, listMsd

In [12]:
folders = os.listdir('/Users/23amrutad/Desktop/data')
imgList = []
ansList = []
for folder_name in folders:
    path = '/Users/23amrutad/Desktop/data/' + folder_name
    folderImgs, folderAns = captureImgWithPath(path, int(folder_name))
    imgList = imgList + folderImgs
    ansList = ansList + folderAns
x = np.array(imgList)
y = np.array(ansList)
x, y = unison_shuffled_copies(x, y)

x = x.reshape(x.shape[0], 28, 28, 1)
y = y.reshape(y.shape[0], 28, 28, 1)
print(x.shape)
print(y.shape)

AttributeError: 'list' object has no attribute 'reshape'

In [11]:
#paths for all of the images

cup_train_path = '/Users/23amrutad/Desktop/data/cup-train-standing'
cup_test_path = '/Users/23amrutad/Desktop/data/cup-test'
apple_train_path = '/Users/23amrutad/Desktop/data/apple-train'
apple_test_path = '/Users/23amrutad/Desktop/data/apple-test'

#
kLabelCup = 0
kLabelApple = 1

#lists to put the images and corresponding answers into for train and test
cupImg, cupAns = captureImgWithPath(cup_train_path, kLabelCup)
appleImg, appleAns = captureImgWithPath(apple_train_path, kLabelApple)

ImagesTrain = np.vstack((cupImg, appleImg))
AnswersTrain = cupAns + appleAns
AnswersTrain = np.array(AnswersTrain)
print('ImagesTrain shape is ', ImagesTrain.shape)
print('AnswersTrain shape is ', AnswersTrain.shape)


cupImg, cupAns = captureImgWithPath(cup_test_path, kLabelCup)
appleImg, appleAns = captureImgWithPath(apple_test_path, kLabelApple)
ImagesTest = np.vstack((cupImg, appleImg))
AnswersTest = cupAns + appleAns
AnswersTest = np.array(AnswersTest)
print('ImageTest shape is ', ImagesTest.shape)
print('AnswersTest shape is ', AnswersTest.shape)


x_train, y_train = unison_shuffled_copies(ImagesTrain, AnswersTrain)
x_test = ImagesTest
y_test = AnswersTest

print(x_train.shape)
print(x_test.shape)
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
print(x_train.shape)
print(x_test.shape)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/23amrutad/Desktop/data/cup-train-standing'

In [4]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
 
input_shape = (28, 28, 1)

 
model = Sequential()
model.add(Conv2D(28, kernel_size=(3,3), input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(16, activation=tf.nn.relu))
model.add(Dropout(0.5))
model.add(Dense(2, activation=tf.nn.softmax))
 
 
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(x=x_train,y=y_train, epochs=60)
model.evaluate(x_test, y_test)

Using TensorFlow backend.


Epoch 1/60
424/424 [==============================] - 0s 780us/step - loss: 0.5840 - accuracy: 0.7052
Epoch 2/60
424/424 [==============================] - 0s 332us/step - loss: 0.3896 - accuracy: 0.8278
Epoch 3/60
424/424 [==============================] - 0s 328us/step - loss: 0.2959 - accuracy: 0.9033
Epoch 4/60
424/424 [==============================] - 0s 336us/step - loss: 0.2887 - accuracy: 0.9057
Epoch 5/60
424/424 [==============================] - 0s 382us/step - loss: 0.2754 - accuracy: 0.9269
Epoch 6/60
424/424 [==============================] - 0s 369us/step - loss: 0.2285 - accuracy: 0.9245
Epoch 7/60
424/424 [==============================] - 0s 323us/step - loss: 0.2161 - accuracy: 0.9410
Epoch 8/60
424/424 [==============================] - 0s 335us/step - loss: 0.1938 - accuracy: 0.9458
Epoch 9/60
424/424 [==============================] - 0s 325us/step - loss: 0.2206 - accuracy: 0.9599
Epoch 10/60
424/424 [==============================] - 0s 334us/step - loss: 0.188

[0.0019641184447788296, 1.0]

In [8]:
pathNewData = '/Users/23amrutad/Desktop/newData/image'
i = 0


previousState = 'empty'
previousImg = None
currentImg = None
cap = cv2.VideoCapture(0)
msdList = []

while(True):
    time.sleep(0.1)
    ret, currentImg = cap.read()
    if (previousImg is not None):
        msd = skimage.measure.compare_mse(currentImg, previousImg)
        msdList.append(msd)
        state, msdList = computeNextState(msdList, previousState, 500)
        #print('the state is ', state)
        if previousState != 'filled' and state == 'filled':
            image_path = pathNewData + '%s'%i +'.jpg'
            print(image_path)
            i = i+1
            cv2.imwrite(image_path, currentImg)
            #print(currentImg.shape, ' is the image shape')
            imgList, _ = preProcessFromPath(currentImg, [], [], 0)
            imgArr = np.array(imgList)
            imgArr = imgArr.reshape(imgArr.shape[0], 28, 28, 1)
            prediction = model.predict(imgArr)
            predictionList = prediction.tolist()
            if predictionList[0][0] > predictionList[0][1]:
                print('it is a cup ', predictionList)
            elif predictionList[0][1] > predictionList[0][0]:
                print('it is an apple ', predictionList)
            else:
                print('confused ', predictionList)
            print(predictionList[0][0])
        previousState = state
    previousImg = currentImg

/Users/23amrutad/Desktop/newData/image0.jpg
it is a cup  [[0.9992721676826477, 0.000727799953892827]]
0.9992721676826477
/Users/23amrutad/Desktop/newData/image1.jpg
it is an apple  [[0.0196347925812006, 0.9803652763366699]]
0.0196347925812006
/Users/23amrutad/Desktop/newData/image2.jpg
it is an apple  [[0.00010817852307809517, 0.9998917579650879]]
0.00010817852307809517
/Users/23amrutad/Desktop/newData/image3.jpg
it is a cup  [[0.995502769947052, 0.004497280344367027]]
0.995502769947052


KeyboardInterrupt: 